# 📘 Enhanced Error Messages in NLSQ> Intelligent, actionable error messages for faster debugging⏱️ **10-15 minutes** | 📊 **Level: ●○○ Beginner** | 🏷️ **Feature Demo**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/05_feature_demos/enhanced_error_messages_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Understand NLSQ's enhanced error diagnostics2. Use error recommendations to fix optimization failures3. Implement programmatic error handling4. Appreciate the improvement over generic error messages---

## 🔬 Feature Overview**What problem does this solve?**Traditional optimization libraries give cryptic errors:```RuntimeError: Optimal parameters not found:CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH```NLSQ provides:- Clear failure reasons in plain English- Specific, actionable recommendations- Detailed diagnostic information- Programmatic error handling capabilities---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
import jax.numpy as jnp
import numpy as np

from nlsq import curve_fit
from nlsq.utils.error_messages import OptimizationError

## Example 1: Maximum Iterations ReachedSee how NLSQ explains common failures with actionable advice.

In [ ]:
def exponential(x, a, b):
    return a * jnp.exp(-b * x)


# Generate data
x = np.linspace(0, 5, 50)
y = 3 * np.exp(-0.5 * x) + np.random.normal(0, 0.1, 50)

try:
    # Force failure with very low max_nfev
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=3)
except OptimizationError as e:
    print('❌ Optimization Failed!')
    print(f'\n{e}')
    print('\n📊 Diagnostic Details:')
    for key, value in e.diagnostics.items():
        print(f'  • {key}: {value}')
    print('\n💡 Recommendations:')
    for i, rec in enumerate(e.recommendations, 1):
        print(f'  {i}. {rec}')

## Example 2: Applying RecommendationsShow how following recommendations leads to success.

In [ ]:
def exponential(x, a, b):
    return a * jnp.exp(-b * x)


x = np.linspace(0, 5, 50)
y = 3 * np.exp(-0.5 * x) + np.random.normal(0, 0.1, 50)

# First attempt: fails
print('🔴 First attempt (max_nfev=3):')
try:
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=3)
    print('  ✅ Succeeded (unexpected)')
except OptimizationError as e:
    print(f'  ❌ Failed: {e.reasons[0] if e.reasons else "Unknown"}')
    print(f'  💡 Recommendation: {e.recommendations[0] if e.recommendations else "Increase max_nfev"}')

# Second attempt: apply recommendation
print('\n🟢 Second attempt (max_nfev=100):')
try:
    popt, pcov = curve_fit(exponential, x, y, p0=[1, 1], max_nfev=100)
    print(f'  ✅ Success! Fitted: a={popt[0]:.3f}, b={popt[1]:.3f}')
    print('  📈 True parameters: a=3.000, b=0.500')
except OptimizationError as e:
    print(f'  ❌ Still failed: {e.reasons[0]}')

## Example 3: Programmatic Error HandlingUse error diagnostics to implement automatic retry strategies.

In [ ]:
def gaussian(x, amp, mu, sigma):
    return amp * jnp.exp(-((x - mu)**2) / (2*sigma**2))


x = np.linspace(-5, 5, 100)
y = 2 * np.exp(-((x - 1)**2) / (2*0.5**2))

try:
    popt, pcov = curve_fit(gaussian, x, y, p0=[1, 0, 1], max_nfev=2)
except OptimizationError as e:
    print('📊 Analyzing Error Diagnostics:')
    print(f'  • Number of reasons: {len(e.reasons)}')
    print(f'  • Number of recommendations: {len(e.recommendations)}')

    # Programmatic decision making
    if any('maximum' in r.lower() for r in e.reasons):
        print('\n🔧 Auto-fix strategy: Increase max_nfev')
        try:
            popt, pcov = curve_fit(gaussian, x, y, p0=[1, 0, 1], max_nfev=200)
            print('  ✅ Auto-retry succeeded!')
            print(f'     Fitted: amp={popt[0]:.2f}, mu={popt[1]:.2f}, sigma={popt[2]:.2f}')
        except OptimizationError:
            print('  ❌ Auto-retry failed')

## Example 4: Before vs After ComparisonSee the dramatic improvement in error message quality.

In [ ]:
def difficult(x, a, b, c):
    return a * jnp.sin(b * x) * jnp.exp(-c * x)


x = np.linspace(0, 10, 50)
y = 2 * np.sin(3 * x) * np.exp(-0.5 * x)

print('🔴 OLD ERROR (before enhancement):')
print('  "RuntimeError: Optimal parameters not found:')
print('   CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH"')
print('\n  😕 Not helpful! What should I do?')

print('\n🟢 NEW ERROR (with enhancement):')
try:
    popt, pcov = curve_fit(difficult, x, y, p0=[1, 1, 1], max_nfev=3)
except OptimizationError as e:
    print(f'\n{e}')

print('\n  ✅ Much better! Clear diagnostics and actionable steps!')

## 💡 Key Insights1. **Enhanced errors** provide clear explanations instead of cryptic codes2. **Diagnostics** give detailed information about what went wrong3. **Recommendations** are specific and actionable4. **Programmatic access** enables automatic error recovery5. **Much faster debugging** compared to traditional error messages---## 📚 Error Types DetectedNLSQ provides enhanced messages for:- Maximum iterations/evaluations reached- Convergence failures- Numerical instabilities- Invalid initial parameters- Singular covariance matrices- And more...---## 🎓 Best Practices- Always catch `OptimizationError` for robust code- Use `e.diagnostics` for detailed troubleshooting- Implement retry logic using `e.recommendations`- Log error details for debugging production issues- Share error messages when asking for help (they're now useful!)---